### 企查查

### 1.按照红头文件匹配

In [1]:
import jieba
import pandas as pd

In [2]:
def get_stop_words():
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])

In [3]:
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']

    
def hadle_official_info():
    '''讲地市的红头文件分词'''
    df = pd.read_csv('./redhead.csv',delimiter='$',header=-1)
    df.head()
    df[1] = df[1].apply(lambda x:strQ2B(x))
    jieba.set_dictionary('data/dict.txt.small')
    x_cut = [] #分词后,去除停用词的语料库

    # 获取停用词列表
    stopwords = get_stop_words()
    cut_res = df[1].apply(cut)
    df[1] = cut_res
    return df

official_info_df = hadle_official_info()

Building prefix dict from /home/lj/ideaprojects/DeepLearning/data/dict.txt.small ...
Dumping model to file cache /tmp/jieba.ube6b3622f3ed3770dfb4b3dbec1af2a2.cache
Loading model cost 0.270 seconds.
Prefix dict has been built succesfully.


In [4]:
official_info_df

,0,1
0,济南,信息技术/智能/制造/高端/装备/量子/科技/生物/医药/先进/材料/产业/金融/现代/物流...
1,青岛,西海岸/新区/蓝谷/核心区/高新区/胶东/临空/经济/示范区/胶州湾/青岛/老城区/有机/更...
2,烟台,烟台/经济/技术开发区/烟台/高新技术/产业/开发区/烟台/东部/产城/融合/发展/示范区/...
3,淄博,国家/省级/经济/技术开发区/高新技术/产业/开发区/海关/特殊/监管/区/新/能源/电池/...
4,枣庄,人工智能/信息技术/新/能源/新/材料/医养/健康
5,东营,航空航天/石化/装备/新/能源/文化/旅游
6,潍坊,虚拟/现实/人工智能/新/能源/电池/高端/装备/制造/生物/基/材料/信息技术/现代/种业
7,济宁,第三代/半导体/生命/健康/信息技术/文化/旅游/生物/医药
8,泰安,人工智能/生命/健康/信息技术/高端/装备/制造/文化/旅游/体育/新/能源
9,威海,生命/健康/前沿/新/材料/医疗/器械/海洋生物/时尚/创意


In [5]:
handle_desc("我爱你中国,哈哈,呵呵,中国")

['爱', '中国', '呵呵', '中国']

In [6]:
official_info_df.to_csv('redhead2.csv',index=0)

In [7]:
official_info_df = pd.read_csv('redhead2.csv',index_col=0)
official_info_df.index
official_info_df.loc['青岛','1']
type(official_info_df.loc['青岛','1'])

str

In [8]:
def judge_by_official(desc,location):
    '''匹配红头文件, 存在则判定为新动能
       -1 : 未找到红头文件
       0: 匹配失败
       1: 新动能匹配成功
    '''
    try:
        print(location)
        official_info_df.loc[location]
    except KeyError: 
        return -1
    official_info = official_info_df.loc[location,'1']
    official_info = official_info.split('/')
    crossinfo = [w for w in desc if w in official_info]
    print('pattern ',location, ' ',len(crossinfo),'字符')
    if len(crossinfo)>1:
        return 1
    else:
        return 0

desc = '青岛达能环保设备股份有限公司，成立于2006年10月，注册资金6300万元，占地面积15万平方米，生产建筑面积5.5万平方米，制造设备齐全，生产装备达到国内同行业领先水平，目前已具备年产值10亿元人民币的生产能力。青岛达能环保设备股份有限公司属锅炉辅机、环保设备制造行业。主要从事电站锅炉干式除渣系统产品、湿式除渣系统产品及烟气余热利用系统产品的研发、生产与销售。青岛达能环保设备股份有限公司为集研发、生产、销售为一体的国家高新技术企业、国家AAA级信用企业、山东省战略新兴产业重点企业、青岛企业技术中心、青岛民营科技创新先进单位。获得国家民营科技发展贡献奖、国家重点新产品、山东优秀节能成果奖等项目的荣誉称号和奖励。随着公司新业务的扩展，现面向社会诚招以下人才'
desc = handle_desc(desc)
location = '青岛'
judge_by_official(desc,location)

青岛
pattern  青岛   16 字符


1

### 2. LDA匹配

In [9]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
neg_cut = pd.read_csv('qichacha_cut.csv',index_col=0)
neg_cut['经营范围'] = neg_cut['经营范围'].apply(lambda x:x.split(',')).values
neg_df = pd.DataFrame({'公司名称':neg_cut['公司名'].apply(lambda x:x[:-2]),
                    '经营范围':neg_cut['经营范围']})
neg_df['企业分类'] = '旧动能'
neg_df = neg_df.dropna(how="any",axis=0)

In [10]:
neg_df.head()

,公司名称,经营范围,企业分类
0,刘向岗,"[批发, 零售]",旧动能
1,安丘市兴建新型装饰材料制品厂,"[加工, 销售]",旧动能
2,安丘市浩红小吃铺,"[小吃, 服务, 依法, 须, 批准, 项目, 相关, 部门, 批准, 后方, 开展, 经营...",旧动能
3,滨州市滨城区建利纸制品批发部,"[纸制品, 需, 许可, 经营, 许可证, 经营]",旧动能
4,滨州市滨城区倩倩化妆品店,[化妆品],旧动能


In [11]:
pos_df = pd.read_excel("pos.xlsx")
pos_df = pd.DataFrame({"公司名称":pos_df["公司名称"],
                       "经营范围":pos_df["经营范围"],
                       "企业分类":pos_df["企业分类"]})
pos_df = pos_df.dropna(how="any",axis=0)

In [12]:
# pos_desc = pos_df["经营范围"].dropna()
pos_df["经营范围"] = pos_df["经营范围"].apply(cut)
pos_df["经营范围"] = pos_df["经营范围"].apply(lambda x:x.split("/"))
# y = np.concatenate((np.ones(pos_desc.shape[0]),np.zeros(x_cut.shape[0])),axis=0)
# x_cut = np.concatenate((pos_desc,x_cut),axis=0)
pos_df.head()

,公司名称,经营范围,企业分类
0,济南二机床集团有限公司技术中心,"[道路, 货物, 运输, 普通, 货运, 三类, 汽车, 维修, 车身, 清洁, 维护, 第...",高端装备制造
1,浪潮集团有限公司技术中心,"[商用, 密码, 产品, 开发, 生产, 销售, 有效, 期限, 许可证, 为准, 计算机,...",信息产业
2,鲁南制药集团股份有限公司技术中心,"[生产, 销售, 原料药, 包装, 物品, 依法, 须经, 批准, 项目, 相关, 部门, ...",医养健康
3,济南轻骑摩托车有限公司技术中心,"[摩托车, 零配件, 设计, 开发, 生产, 销售, 摩托车, 技术, 咨询, 服务, 引进...",高端装备制造
4,万华化学集团股份有限公司技术中心,"[安全, 生产, 许可证, 范围, 化学, 危险品, 生产, 许可证, 范围, 铁路, 专用...",高端化工


In [15]:
# 非SVM数据处理
new_pos = pos_df.copy()
for i in range(50):
    pos_df = pd.concat((new_pos,pos_df))
pos_df.shape

(8313, 3)

In [20]:
# 全部训练集
df = pd.concat((pos_df,neg_df))
print(df.shape)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["企业分类"] = le.fit_transform(df["企业分类"])
print(le.classes_)
df.head()

(9463, 3)
['信息产业' '医养健康' '文化产业' '旅游产业' '旧动能' '海洋经济' '现代农业' '现代金融' '能源原材料' '能源新材料'
 '高端化工' '高端装备制造']


,公司名称,经营范围,企业分类
0,济南二机床集团有限公司技术中心,"[道路, 货物, 运输, 普通, 货运, 三类, 汽车, 维修, 车身, 清洁, 维护, 第...",11
1,浪潮集团有限公司技术中心,"[商用, 密码, 产品, 开发, 生产, 销售, 有效, 期限, 许可证, 为准, 计算机,...",0
2,鲁南制药集团股份有限公司技术中心,"[生产, 销售, 原料药, 包装, 物品, 依法, 须经, 批准, 项目, 相关, 部门, ...",1
3,济南轻骑摩托车有限公司技术中心,"[摩托车, 零配件, 设计, 开发, 生产, 销售, 摩托车, 技术, 咨询, 服务, 引进...",11
4,万华化学集团股份有限公司技术中心,"[安全, 生产, 许可证, 范围, 化学, 危险品, 生产, 许可证, 范围, 铁路, 专用...",10


In [21]:
def getLDA(topic_num):
    from gensim.models import LdaMulticore
    from gensim.corpora import Dictionary
    # gensim
    dictionary = Dictionary(df["经营范围"])
    print(dictionary)
    corpus = [dictionary.doc2bow(doc) for doc in df["经营范围"]]
    lda = LdaMulticore(corpus=corpus,  # LDA训练语料
                   id2word=dictionary, # id到单词的映射表
                   num_topics=topic_num)      # LDA主题数量 
    return lda,corpus,dictionary
topic_num = 25
lda,corpus,dictionary = getLDA(topic_num)
lda.print_topics()

Dictionary(3025 unique tokens: ['GC2', '三类', '专用', '业务', '为准']...)


[(21,
  '0.026*"销售" + 0.024*"批准" + 0.021*"生产" + 0.020*"技术" + 0.015*"设备" + 0.015*"相关" + 0.014*"项目" + 0.014*"活动" + 0.013*"经营" + 0.012*"依法"'),
 (16,
  '0.030*"批准" + 0.026*"销售" + 0.021*"经营" + 0.019*"服务" + 0.017*"开展" + 0.016*"相关" + 0.014*"港口" + 0.014*"活动" + 0.014*"种植" + 0.014*"许可证"'),
 (13,
  '0.035*"服务" + 0.030*"销售" + 0.030*"批准" + 0.027*"生产" + 0.021*"设备" + 0.020*"相关" + 0.018*"许可证" + 0.018*"技术" + 0.015*"经营" + 0.015*"项目"'),
 (4,
  '0.037*"批准" + 0.034*"销售" + 0.029*"经营" + 0.022*"相关" + 0.021*"服务" + 0.020*"开展" + 0.020*"生产" + 0.019*"活动" + 0.018*"部门" + 0.018*"许可证"'),
 (8,
  '0.027*"经营" + 0.026*"批准" + 0.022*"设备" + 0.016*"技术" + 0.014*"工程" + 0.013*"开展" + 0.013*"部门" + 0.013*"销售" + 0.012*"后方" + 0.012*"机械"'),
 (17,
  '0.041*"销售" + 0.028*"批准" + 0.021*"设备" + 0.020*"相关" + 0.018*"服务" + 0.017*"经营" + 0.017*"项目" + 0.017*"工程" + 0.016*"进出口" + 0.015*"活动"'),
 (22,
  '0.042*"批准" + 0.037*"设备" + 0.028*"机械" + 0.024*"相关" + 0.023*"开展" + 0.023*"项目" + 0.023*"经营" + 0.023*"活动" + 0.022*"部门" + 0.021*"依法"'),
 (19,
  '0.046*"批准

In [22]:
def get_topic_distribute(lda,doc):
    return lda[doc]

def get_all_doc_topic():
    lda_matrix = np.zeros((df["经营范围"].shape[0],topic_num),dtype='float64')
    for index,doc in enumerate(corpus):
        topics = get_topic_distribute(lda,corpus[index])
        for topic_tuple in topics:
            lda_matrix[index,topic_tuple[0]] = topic_tuple[1]
    return lda_matrix

print(get_topic_distribute(lda,corpus[0]))
print(len(corpus))
# 所有样本主题分布
all_topic = get_all_doc_topic()
# x_neg = all_topic[:1150]
# x_pos = all_topic[1150:]
# # svm正例
# y_neg = np.zeros(x_neg.shape[0])

[(8, 0.99428564)]
9463


In [40]:
x = all_topic
y = pd.get_dummies(df["企业分类"])
print(x.shape,y.shape)
print(len(le.classes_)) 

(9463, 25) (9463, 12)
12


In [48]:
import keras 
from keras import models
from keras import layers
network = models.Sequential()
# input_shape : 输入张量的形状, (28*28,)表示1维度向量
network.add(layers.Dense(32,activation='relu',input_shape=(x.shape[1],)))
network.add(layers.Dense(len(le.classes_),activation='softmax'))
# 4. 编译
network.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

# 5. 训练模型
network.fit(x,y,epochs=60,batch_size=2)

Epoch 1/60
9463/9463 [==============================] - 3s 313us/step - loss: 1.8044 - acc: 0.3931
Epoch 2/60
9463/9463 [==============================] - 3s 295us/step - loss: 1.5229 - acc: 0.4531
Epoch 3/60
9463/9463 [==============================] - 3s 295us/step - loss: 1.4202 - acc: 0.4881
Epoch 4/60
9463/9463 [==============================] - 3s 295us/step - loss: 1.3367 - acc: 0.5231
Epoch 5/60
9463/9463 [==============================] - 3s 296us/step - loss: 1.2751 - acc: 0.5396
Epoch 6/60
9463/9463 [==============================] - 3s 294us/step - loss: 1.2310 - acc: 0.5506
Epoch 7/60
9463/9463 [==============================] - 3s 299us/step - loss: 1.2072 - acc: 0.5590
Epoch 8/60
9463/9463 [==============================] - 3s 298us/step - loss: 1.1719 - acc: 0.5773
Epoch 9/60
9463/9463 [==============================] - 3s 301us/step - loss: 1.1491 - acc: 0.5836
Epoch 10/60
9463/9463 [==============================] - 3s 301us/step - loss: 1.1240 - acc: 0.5886
Epoch 11/

In [ ]:
# SVM多分类

from sklearn.model_selection import train_test_split
def train_svm(lda_matrix,label):
    clf = SVC(probability=True)
    clf.fit(lda_matrix, label) 
    return clf


clfs = {}
for col in label_df.columns:
    print(col)
    df = pd.concat((topic_df,label_df[col]),axis=1)
    new_df = df[df[col]==1]
#     print(new_df.shape)
#     print(df.shape)
    # 正例膨胀100倍
    for j in range(100):
        df = pd.concat((df,new_df))
    x = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
    clf = train_svm(x_train,y_train)
    clfs[col] = clf
    print(clf.score(x_test,y_test))

In [ ]:
import pickle
#保存词典
with open('dict_save', 'wb') as f:                     
    picklestring = pickle.dump(dictionary, f)
#保存lda模型
lda.save('lda.model')
#保存svm模型
with open('clfs.pickle', 'wb') as fw:
    pickle.dump(clfs, fw)

### 预测

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import jieba

with open('dict_save', 'rb') as f:
    load_dictionary = pickle.load(f)   # read file and build object
from gensim import models
load_lda = models.ldamodel.LdaModel.load('lda.model')
# 加载svm.pickle
with open('clfs.pickle', 'rb') as fr:
    clfs = pickle.load(fr)


def get_stop_words():
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']
# def predict(dictionary,description,clf):
#     a = dictionary.doc2bow(handle_desc(description))
#     topic_distribute = lda[a]
#     res = clf.predict(topic_distribute)
#     return res

# desc = "重金属污染"
desc = " 计算机及外部设备、电力自动化及工业自动化控制系统、电子产品及通讯设备、电子元器件、计算机软硬件、机房设施、仪器仪表、汽车电器的开发、生产、销售及技术咨询服务;机械工程、塑料注塑模具和注塑件、建筑智能化及建筑节能工程设计与施工;新能源汽车充换电设备的研发、生产、销售和服务;地面卫星接收站、绕线机、铣曲机、衡器的制造、销售;节能技术的研发、运维、服务、转让;光伏发电的设计、研发、建设、维护及技术咨询;售电业务;进出口业务及对外经济技术合作业务。(依法须经批准的项目,经相关部门批准后方可开展经营活动)"
# print(y[0])
topic_num = 25
a = load_dictionary.doc2bow(handle_desc(desc))
topic_distribute = load_lda[a]
print(topic_distribute)
topics = np.zeros((1,topic_num))
print(topics)
for tupl in topic_distribute:
    print(tupl)
    topics[0,tupl[0]]=tupl[1]
print(topics[0,:])
res = {}
for clf_name,clf_model in clfs.items():
    clf_res = clf_model.predict_proba(topics)
    if clf_name != '旧动能':
        res[clf_res[0,:][1]] = clf_name
    print(clf_name+" :",clf_res[0,:])
max_score = sorted(res.keys(),reverse=True)[0]
if max_score<0.5:
    label = "旧动能"
else:
    label = (res[max_score])
print(label)

### 补全地市

In [ ]:
def get_area():
    df2 = pd.read_csv('qichacha.csv',index_col=0)
    return df2['登记机关'].apply(lambda x:x[:2])
    
qichacha_cut['所在区域']=get_area()
qichacha_cut.head()

In [ ]:
qichacha_cut.shape[0]

In [ ]:
SCRIPT_STR('
import pickle
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import jieba

with open('/root/DeepLearning/dict_save', 'rb') as f:
    load_dictionary = pickle.load(f)   # read file and build object
from gensim import models
load_lda = models.ldamodel.LdaModel.load('lda.model')
# 加载svm.pickle
with open('/root/DeepLearning/svm.pickle', 'rb') as fr:
    load_clf = pickle.load(fr)


def get_stop_words():
    with open('/root/DeepLearning/data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']
# def predict(dictionary,description,clf):
#     a = dictionary.doc2bow(handle_desc(description))
#     topic_distribute = lda[a]
#     res = clf.predict(topic_distribute)
#     return res


# print(x_cut[0])
# print(y[0])
topic_num = 25
a = load_dictionary.doc2bow(handle_desc("航空航天"))
topic_distribute = load_lda[a]
print(topic_distribute)
topics = np.zeros((1,topic_num))
print(topics)
for tupl in topic_distribute:
    print(tupl)
    topics[0,tupl[0]]=tupl[1]
print(topics)
res = load_clf.predict(topics)
res[0]
','')

In [3]:
import pandas as pd
pd.read_excel("pos.xlsx").to_csv("pos.csv")